# On Kaggle 

In [ ]:
# !pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 41.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.


In [2]:
from typing import Optional, Tuple, List, Dict, Any
from dataclasses import dataclass, field

import numpy as np
import torch
import wandb
import evaluate
from torch import optim, nn
from datasets import load_dataset
from transformers import SegformerImageProcessor, AutoModelForSemanticSegmentation
from torch.utils.data import DataLoader
import albumentations as A

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'


## 2. Training Configuration

### Configuration Dataclass
Define all hyperparameters and training settings in a structured configuration object.

**Key Parameters:**
- **Batch Size:** 8
- **Epochs:** 6
- **Learning Rate:** 1e-4 with exponential decay (0.9998)
- **Class Weighting:** Background weight = 1.0, Other classes = 3.0 (to handle class imbalance)

In [3]:
@dataclass
class TrainingConfig:
    """
    Training configuration for the model.

    Args:
        batch_size: Batch size for training.
        epochs: Number of epochs to train for.
        learning_rate: Learning rate for the optimizer.
        background_weight: Weight for the background class.
        other_classes_weight: Weight for the other classes.
        lr_decay_rate: Learning rate decay rate.
        seed: Random seed for reproducibility.
        model_name: Name of the model to use.
        project_name: Name of the project for wandb.
        device: Device to use for training.
    """
    batch_size: int = 8
    epochs: int = 6
    learning_rate: float = 1e-4
    background_weight: float = 1.0
    other_classes_weight: float = 3.0
    lr_decay_rate: float = 0.9998
    seed: int = 42
    model_name: str = "nvidia/mit-b2"
    project_name: str = "Clothes segmentation"
    device: Optional[str] = field(
        default_factory=lambda: "cuda" if torch.cuda.is_available() else "cpu"
    )

    def as_dict(self) -> Dict[str, Any]:
        return vars(self)


config = TrainingConfig(
    batch_size=8,
    epochs=6,
    learning_rate=1e-4,
    background_weight=1.0,
    other_classes_weight=3.0,
    lr_decay_rate=0.9998,
    model_name="nvidia/mit-b2",
    project_name="Clothes segmentation",
)
wandb_config = config.as_dict()

In [4]:
# Dataset Loading and Preparation
ds = load_dataset("mattmdjaga/human_parsing_dataset", split="train[:100%]", num_proc=8)
ds.shuffle(seed=config.seed)

split_ratio: float = 0.006
split_size: int = int(len(ds) * split_ratio)
print(f"Split size: {split_size}")
ds_split = ds.train_test_split(test_size=split_ratio, seed=config.seed)
train_ds = ds_split["train"]
test_ds = ds_split["test"]

README.md: 0.00B [00:00, ?B/s]

Setting num_proc from 8 to 2 for the train split as it only contains 2 shards.


Generating train split:   0%|          | 0/17706 [00:00<?, ? examples/s]

Split size: 106


### Class Labels
The model segments images into 18 distinct classes:

In [5]:
id2label: Dict[str, str] = {
    "0": "Background",
    "1": "Hat",
    "2": "Hair",
    "3": "Sunglasses",
    "4": "Upper-clothes",
    "5": "Skirt",
    "6": "Pants",
    "7": "Dress",
    "8": "Belt",
    "9": "Left-shoe",
    "10": "Right-shoe",
    "11": "Face",
    "12": "Left-leg",
    "13": "Right-leg",
    "14": "Left-arm",
    "15": "Right-arm",
    "16": "Bag",
    "17": "Scarf",
}
label2id: Dict[str, str] = {v: k for k, v in id2label.items()}
num_labels: int = len(id2label)

### Base Model: SegFormer-B2 (nvidia/mit-b2)

**SegFormer** is a transformer-based semantic segmentation model that uses:
- **Mix Transformer (MiT)** encoder for hierarchical feature extraction
- **Lightweight All-MLP decoder** for efficient segmentation
- **No positional encoding required** - works with arbitrary input sizes

**Model Specifications:**
- Pretrained on ImageNet-1K
- Fine-tuned for 18-class segmentation task
- Parameters: ~27M (B2 variant)

In [6]:
# Model and Tokenizer Initialization
tokenizer = SegformerImageProcessor.from_pretrained(config.model_name)
model = AutoModelForSemanticSegmentation.from_pretrained(
    config.model_name, num_labels=num_labels, id2label=id2label, label2id=label2id
).to(config.device)

preprocessor_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/utils/deprecation.py:172: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/99.0M [00:00<?, ?B/s]

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b2 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 5. Data Augmentation

### Augmentation Pipeline using Albumentations

**Training Augmentations:**
- Horizontal flip (50% probability)
- Image quality degradation (downscaling, Gaussian noise)
- Color transformations (brightness, contrast, hue, saturation)
- Occlusion simulation (pixel dropout, gravel)

**Note:** Augmentations are applied to both images and masks simultaneously.


In [7]:
# Image Transformations
img_transforms = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.OneOf(
            [
                A.Downscale(p=0.1, scale_min=0.4, scale_max=0.6),
                A.GaussNoise(p=0.2),
            ],
            p=0.1,
        ),
        A.OneOf(
            [
                A.RandomBrightnessContrast(p=0.2),
                A.ColorJitter(p=0.2),
                A.HueSaturationValue(p=0.2),
            ],
            p=0.1,
        ),
        A.OneOf([A.PixelDropout(p=0.2), A.RandomGravel(p=0.2)], p=0.15),
    ]
)

/tmp/ipykernel_20/1038826196.py:7: UserWarning: Argument(s) 'scale_min, scale_max' are not valid for transform Downscale
  A.Downscale(p=0.1, scale_min=0.4, scale_max=0.6),


model.safetensors:   0%|          | 0.00/98.9M [00:00<?, ?B/s]

In [8]:
# Dataset Transformation Functions
def train_transforms(example_batch: Dict[str, Any]) -> Dict[str, torch.Tensor]:
    """
    Transform the dataset for training.

    Args:
        example_batch: Batch of examples from the dataset.

    Returns:
        A dictionary of the inputs to the model.
    """
    trans = [
        img_transforms(image=np.array(x), mask=np.array(m))
        for x, m in zip(example_batch["image"], example_batch["mask"])
    ]
    images = [x["image"] for x in trans]
    labels = [x["mask"] for x in trans]
    inputs = tokenizer(images, labels)
    return inputs


def val_transforms(example_batch: Dict[str, Any]) -> Dict[str, torch.Tensor]:
    """
    Transform the dataset for validation.

    Args:
        example_batch: Batch of examples from the dataset.

    Returns:
        A dictionary of the inputs to the model.
    """
    images = [x for x in example_batch["image"]]
    labels = [x for x in example_batch["mask"]]
    inputs = tokenizer(images, labels)
    return inputs


train_ds.set_transform(train_transforms)
test_ds.set_transform(val_transforms)

## 6. Evaluation Metrics

### Mean Intersection over Union (mIoU)

The primary metric for semantic segmentation that measures overlap between predicted and ground truth segments.

**Additional Metrics:**
- **Mean Accuracy:** Per-class accuracy averaged across all classes
- **Overall Accuracy:** Pixel-wise accuracy across entire image

In [9]:
# Metrics and Validation
metric = evaluate.load("mean_iou")


@torch.no_grad()
def compute_metrics(eval_pred: Tuple[torch.Tensor, torch.Tensor]) -> Dict[str, Any]:
    """
    Compute the IOU and accuracy metrics.

    Args:
        eval_pred: Tuple of logits and labels.

    Returns:
        A dictionary of the metrics.
    """
    logits, labels = eval_pred
    logits_tensor = logits.argmax(dim=1)

    pred_labels = logits_tensor

    mets = metric.compute(
        predictions=pred_labels,
        references=labels,
        num_labels=num_labels,
        ignore_index=255,
        reduce_labels=False,
    )
    for key, value in mets.items():
        if type(value) is np.ndarray:
            mets[key] = value.tolist()

    return mets


@torch.no_grad()
def validation(
    model: nn.Module, val_loader: DataLoader
) -> Tuple[torch.Tensor, torch.Tensor, List[float]]:
    """
    Perform validation on the model.

    Args:
        model: Model to validate.
        val_loader: Validation data loader.

    Returns:
        Tuple of logits, labels, and validation losses.
    """
    model.eval()
    val_losses = []
    all_labels = []
    all_logits = []

    for i, batch in enumerate(val_loader, 1):
        inputs = batch["pixel_values"].to(config.device)
        labels = batch["labels"].to(config.device)

        outputs = model(inputs)
        logits_tensor = nn.functional.interpolate(
            outputs.logits,  # Detach to avoid saving gradients
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        )

        loss = loss_func(logits_tensor, labels)
        val_losses.append(loss.item())

        # Storing logits and labels as CPU tensors to save GPU memory
        all_labels.append(labels.cpu())
        all_logits.append(logits_tensor.cpu())

    # Concatenate all logits and labels
    logits = torch.cat(all_logits, dim=0).to(config.device)
    labels = torch.cat(all_labels, dim=0).to(config.device)

    model.train()
    return logits, labels, val_losses

In [10]:
def save_model(model: nn.Module, tokenizer: SegformerImageProcessor, name: str) -> None:
    model.save_pretrained(name)
    tokenizer.save_pretrained(name)

In [11]:
# DataLoader, Optimizer, and Scheduler
train_loader = DataLoader(train_ds, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(test_ds, batch_size=config.batch_size, shuffle=False)
optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate)
lambda1 = lambda step: config.lr_decay_rate**step
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

weights = torch.tensor(
    [config.background_weight] + [config.other_classes_weight] * 17
).to(config.device)
loss_func = nn.CrossEntropyLoss(weight=weights)

In [12]:
from kaggle_secrets import UserSecretsClient
import os, wandb

user_secrets = UserSecretsClient()
WANDB_API_KEY = user_secrets.get_secret("WANDB_API_KEY")

wandb.login(key=WANDB_API_KEY)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abdelrahmantawfik (abdelrahman-tawfik) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [13]:
running_loss: List[float] = []

# Generating a run name using the configuration parameters
run_name = "_".join([f"{key}_{value}" for key, value in wandb_config.items()])
t_steps = 0

# Initialize wandb with the configuration
wandb.init(project=config.project_name, config=wandb_config)
wandb.run.name = run_name


wandb: Tracking run with wandb version 0.21.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20251129_153907-ot20omzh
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run bright-sun-3
wandb: ⭐️ View project at https://wandb.ai/abdelrahman-tawfik/Clothes%20segmentation
wandb: 🚀 View run at https://wandb.ai/abdelrahman-tawfik/Clothes%20segmentation/runs/ot20omzh


### Training Process

**Training Strategy:**
- 6 epochs with exponential learning rate decay
- Validation every 600 steps
- Metrics logged: training loss, validation loss, mIoU, mean accuracy, overall accuracy
- Model checkpoints saved at the end of each epoch

**Key Implementation Details:**
- Bilinear upsampling of logits to match label dimensions
- Weighted cross-entropy loss for class imbalance
- Gradient accumulation not used (direct optimization per batch)

In [14]:
for epoch in range(config.epochs):
    for i, batch in enumerate(train_loader):
        optimizer.zero_grad()
        inputs = batch["pixel_values"].to(config.device)
        labels = batch["labels"].to(config.device)
        outputs = model(inputs)
        # The models predicts small masks, so we need to upsample them to the correct size like in inference
        logits_tensor = nn.functional.interpolate(
            outputs.logits,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        )
        loss = loss_func(logits_tensor, labels)
        running_loss.append(loss.item())

        loss.backward()
        optimizer.step()
        scheduler.step()

        if t_steps % 600 == 0:
            wandb_logs = {}

            last_lr = scheduler.get_last_lr()[0]

            logits_tensor, labels, val_losses = validation(model, val_loader)
            r_loss = sum(running_loss) / len(running_loss)
            val_loss = sum(val_losses) / len(val_losses)

            mets = compute_metrics((logits_tensor, labels))

            wandb_logs["training_loss"] = r_loss
            wandb_logs["val_loss"] = val_loss

            for key, value in mets.items():
                if isinstance(value, float):
                    wandb_logs[key] = value

            print(f"\nEpoch {epoch} Iteration {i}")
            for key, score in wandb_logs.items():
                print(f"{key}: {score:.3f}")

            print(f"LR: {last_lr}")
            wandb_logs["LR"] = last_lr
            wandb.log(wandb_logs)
            running_loss = []
        t_steps += 1
    save_model(model, tokenizer, f"{epoch}")

/usr/local/lib/python3.11/dist-packages/datasets/features/image.py:371: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")



Epoch 0 Iteration 0
training_loss: 2.943
val_loss: 2.735
mean_iou: 0.043
mean_accuracy: 0.065
overall_accuracy: 0.659
LR: 9.998e-05

Epoch 0 Iteration 600
training_loss: 0.519
val_loss: 0.337
mean_iou: 0.466
mean_accuracy: 0.610
overall_accuracy: 0.928
LR: 8.867324103580549e-05

Epoch 0 Iteration 1200
training_loss: 0.350
val_loss: 0.333
mean_iou: 0.511
mean_accuracy: 0.662
overall_accuracy: 0.932
LR: 7.86451657910988e-05

Epoch 0 Iteration 1800
training_loss: 0.316
val_loss: 0.302
mean_iou: 0.516
mean_accuracy: 0.649
overall_accuracy: 0.936
LR: 6.975116765848157e-05

Epoch 1 Iteration 200
training_loss: 0.298
val_loss: 0.283
mean_iou: 0.526
mean_accuracy: 0.658
overall_accuracy: 0.938
LR: 6.186299362181852e-05

Epoch 1 Iteration 800
training_loss: 0.272
val_loss: 0.272
mean_iou: 0.523
mean_accuracy: 0.672
overall_accuracy: 0.940
LR: 5.486689482520514e-05

Epoch 1 Iteration 1400
training_loss: 0.269
val_loss: 0.290
mean_iou: 0.525
mean_accuracy: 0.664
overall_accuracy: 0.937
LR: 4.866